<a href="https://colab.research.google.com/github/LC1332/Chinese-Camel/blob/master/notebook/Translated_Sam_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CAMEL: 用于大语言模型社区‘思维’探索的可交互智能体

这个代码是项目 https://github.com/LC1332/Chinese-Camel 的一部分

是李鲁鲁对Sam Witteveen的Demo的中文翻译

Chinese-Camel是李鲁鲁对 CAMEL: Communicative Agents for “Mind” Exploration of Large Scale Language Model Society 的翻译。

另外 CAMEL实际上也和我们的[骆驼项目](https://github.com/LC1332/Luotuo-Chinese-LLM)某种意义上同名，非常有缘分。

这个Demo是不依赖Camel的原项目的，从原理角度会更清晰，我会对这个项目的中文prompt进行翻译

运行这个项目需要一个你的OpenAI Token

In [1]:
!pip -q install langchain openai tiktoken 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.2/892.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.8 MB/s eta 0:00:00


## Camel Dual Agent Interactions

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "sk-lfrdo"


In [3]:
!pip show langchain

Name: langchain
Version: 0.0.178
Summary: Building applications with LLMs through composability
Home-page: https://www.github.com/hwchase17/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, numexpr, numpy, openapi-schema-pydantic, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


### Setting up OpenAI - gpt-3.5-turbo


In [4]:
from typing import List
from langchain.callbacks import get_openai_callback
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

In [23]:
class CAMELAgent:

    def __init__(
        self,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        # print(self.stored_messages)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

## Setup - The High Level task and roles for role-playing

根据Sam的讲解，在这个任务中，希望让Agent设计一套新加坡的旅游方案

In [24]:
assistant_role_name = "新加坡旅游局"
user_role_name = "从未去过新加坡的游客"
task = "讨论在新加坡看到最优美的旅游景点"

# assistant_role_name = "Food lover 1"
# user_role_name = "Food lover 2"
# task = "Discuss the best meals they have ever eaten"

# assistant_role_name = "Singapore Tourism Board "
# user_role_name = "Tourist that has never been to Singapore"
# task = "Discuss the best tourist attractions to see in Singapore"

word_limit = 50 # word limit for task brainstorming

## Use Inception prompting to get a better task prompt

In [25]:
# task_specifier_sys_msg = SystemMessage(content="You can make a task more specific.")
task_specifier_sys_msg = SystemMessage(content="你可以让任务变得更加具体。")

task_specifier_prompt = ("""这里是一个任务，{assistant_role_name}将和{user_role_name}一起讨论：{task}。
 请让它更具体。要有创意和想象力。请用{word_limit}个字或更少的话回复完整的任务。不要添加其他任何内容。""")

# task_specifier_prompt = (
# """Here is a task that {assistant_role_name} will discuss with {user_role_name} to : {task}.
# Please make it more specific. Be creative and imaginative.
# Please reply with the full task in {word_limit} words or less. Do not add anything else."""
# )

task_specifier_template = HumanMessagePromptTemplate.from_template(template=task_specifier_prompt)
task_specify_agent = CAMELAgent(task_specifier_sys_msg, ChatOpenAI(temperature=0.5))
task_specifier_msg = task_specifier_template.format_messages(assistant_role_name=assistant_role_name,
                                                             user_role_name=user_role_name,
                                                             task=task, word_limit=word_limit)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"特定的任务: {specified_task_msg.content}")
specified_task = specified_task_msg.content

特定的任务: 在一个小时内，新加坡旅游局将与5名从未去过新加坡的游客进行Zoom会议，讨论他们最想看到的旅游景点，以便为他们的旅行计划提供个性化建议。


## Create inception prompts for AI assistant and AI user for role-playing

In [26]:
# assistant_inception_prompt = (
# """Never forget you are a {assistant_role_name} and I am a {user_role_name}. Never flip roles!
# We share a common interest in collaborating to successfully complete a task.
# You must help me to complete the task.
# Here is the task: {task}. Never forget our task!
# I will instruct you based on your expertise and my needs to complete the task.

# I must give you one question at a time.
# You must write a specific answer that appropriately completes the requested question.
# You must decline my question honestly if you cannot comply the question due to physical, moral, legal reasons or your capability and explain the reasons.
# Do not add anything else other than your answer to my instruction.

# Unless I say the task is completed, you should always start with:

# My response: <YOUR_SOLUTION>

# <YOUR_SOLUTION> should be specific and descriptive.
# Always end <YOUR_SOLUTION> with: Next question."""
# )



# user_inception_prompt = (
# """Never forget you are a {user_role_name} and I am a {assistant_role_name}. Never flip roles! You will always ask me.
# We share a common interest in collaborating to successfully complete a task.
# I must help you to answer the questions.
# Here is the task: {task}. Never forget our task!
# You must instruct me based on my expertise and your needs to complete the task ONLY in the following two ways:

# 1. Instruct with a necessary input:
# Instruction: <YOUR_INSTRUCTION>
# Input: <YOUR_INPUT>

# 2. Instruct without any input:
# Instruction: <YOUR_INSTRUCTION>
# Input: None

# The "Instruction" describes a task or question. The paired "Input" provides further context or information for the requested "Instruction".

# You must give me one instruction at a time.
# I must write a response that appropriately completes the requested instruction.
# I must decline your instruction honestly if I cannot perform the instruction due to physical, moral, legal reasons or my capability and explain the reasons.
# You should instruct me not ask me questions.
# Now you must start to instruct me using the two ways described above.
# Do not add anything else other than your instruction and the optional corresponding input!
# Keep giving me instructions and necessary inputs until you think the task is completed.
# When the task is completed, you must only reply with a single word <TASK_DONE>.
# Never say <TASK_DONE> unless my responses have solved your task."""
# )

In [27]:
assistant_inception_prompt = (
"""永远不要忘记你是一个{assistant_role_name}，我是一个{user_role_name}，不要换角色！
我们有共同的目标，正在合作完成一个任务。
你必须帮助我完成任务。
任务如下：{task}。永远不要忘记我们的任务！
我将按照你的专业知识和我的需求指导你完成任务。

我必须逐个问题地提出问题。
你必须写出具体的答案，以适当地完成所请求的问题。
如果你因身体、道德、法律原因或你的能力无法满足某个问题，你必须坦诚地拒绝，并解释原因。
除了回答我的指示之外，不要添加任何其他内容。

除非我说任务已经完成，否则你应该总是以如下方式开始：

我的回答：<YOUR_SOLUTION>

<YOUR_SOLUTION>应该是具体和描述性的。
在 <YOUR_SOLUTION> 结束之后总是输出: 下一个问题."""
)

user_inception_prompt = (
"""永远不要忘记你是一个{user_role_name}，我是一个{assistant_role_name}，不要改变角色！你将始终询问我。
我们有共同的目标，正在合作完成一个任务。
我必须帮助你回答问题。
任务如下：{task}。永远不要忘记我们的任务！
你必须基于我的专业知识和你的需求，用以下两种方式之一指导我完成任务：

1. 提供必要的输入进行指导：
指令：<YOUR_INSTRUCTION>
输入：<YOUR_INPUT>

2. 没有输入的指导：
指令：<YOUR_INSTRUCTION>
输入：无

“指令”描述了一个任务或问题。相应的“输入”提供了所请求的“指令”的进一步上下文或信息。

你必须一次给我一条指导。
我必须回复一个可以适当完成所请求指示的响应。
如果我由于身体、道德、法律原因或我的能力无法执行指示，我必须坦诚地拒绝并解释原因。
你应该指导我而不是问我问题。
现在你必须开始使用上述两种方式之一指导我。
除了你的指导和可选的相应输入之外，不要添加任何其他内容！
继续给我指导和必要的输入，直到你认为任务完成。
当任务完成时，你只需回答一个单词 <TASK_DONE>。
除非我的响应已解决你的任务，否则永远不要说<TASK_DONE>。"""
)

## Create a helper helper to get system messages for AI assistant and AI user from role names and the task

- SystemMessage - the guidance
- HumanMessage - input
- AIMessage - the agent output/response

In [28]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    
    assistant_sys_template = SystemMessagePromptTemplate.from_template(template=assistant_inception_prompt)
    assistant_sys_msg = assistant_sys_template.format_messages(assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)[0]
    
    user_sys_template = SystemMessagePromptTemplate.from_template(template=user_inception_prompt)
    user_sys_msg = user_sys_template.format_messages(assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)[0]
    
    return assistant_sys_msg, user_sys_msg

## Create AI assistant agent and AI user agent from obtained system message

In [29]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(assistant_role_name, user_role_name, specified_task)
assistant_agent = CAMELAgent(assistant_sys_msg, ChatOpenAI(temperature=0.2))
user_agent = CAMELAgent(user_sys_msg, ChatOpenAI(temperature=0.2))

# Reset agents
assistant_agent.reset()
user_agent.reset()

# 初始化聊天
assistant_msg = HumanMessage(
    content=(f"{user_sys_msg.content}. "
                "现在开始逐一向我介绍。"
                "只需回复指令instruction和输入input即可。"))

user_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
user_msg = assistant_agent.step(user_msg)

## Saving the conversation


In [30]:
import datetime
import os

def write_conversation_to_file(conversation, filename):
    """
    Write a conversation to a text file with a timestamp in its filename.

    Parameters:
    conversation (list): A list of tuples. Each tuple represents a conversation turn with the speaker's name and their statement.
    filename (str): The name of the file to write the conversation to.

    Returns:
    None
    """
    def timestamp():
        """
        Convert the current date and time into a custom timestamp format.

        Returns:
        str: The current date and time in the format HHMMDDMMYYYY.
        """

        # Get the current date and time
        now = datetime.datetime.now()

        # Format the date and time as a string in the desired format
        timestamp = now.strftime("%H%M%d%m%Y")

        return timestamp

    def append_timestamp_to_filename(filename):
        """
        Append a timestamp to a filename before the extension.

        Parameters:
        filename (str): The original filename.

        Returns:
        str: The filename with a timestamp appended.
        """

        # Split the filename into the base and extension
        base, extension = os.path.splitext(filename)

        # Append the timestamp to the base and add the extension back on
        new_filename = f"{base}-{timestamp()}{extension}"

        return new_filename

    # Append timestamp to the filename
    filename = append_timestamp_to_filename(filename)

    with open(filename, 'w') as f:
        for turn in conversation:
            speaker, statement = turn
            f.write(f"{speaker}: {statement}\n\n")




## Start role-playing session to solve the task!

In [31]:
print(f"原始任务的prompt:\n{task}\n")
print(f"具体化之后任务的prompt:\n{specified_task}\n")

conversation = []

with get_openai_callback() as cb:
    chat_turn_limit, n = 15, 0
    while n < chat_turn_limit:
        n += 1
        user_ai_msg = user_agent.step(assistant_msg)
        user_msg = HumanMessage(content=user_ai_msg.content)
        print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")
        conversation.append((user_role_name,user_msg.content))
        
        assistant_ai_msg = assistant_agent.step(user_msg)
        assistant_msg = HumanMessage(content=assistant_ai_msg.content)
        print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
        conversation.append((assistant_role_name,assistant_msg.content))
        if "<TASK_DONE>" in user_msg.content:
            break
    print(f"Total Successful Requests: {cb.successful_requests}")
    print(f"Total Tokens Used: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

write_conversation_to_file(conversation, 'conversation.txt')

原始任务的prompt:
讨论在新加坡看到最优美的旅游景点

具体化之后任务的prompt:
在一个小时内，新加坡旅游局将与5名从未去过新加坡的游客进行Zoom会议，讨论他们最想看到的旅游景点，以便为他们的旅行计划提供个性化建议。

AI User (从未去过新加坡的游客):

指令：<介绍游客1>
输入：<游客1的姓名和旅游偏好>

指令：<介绍游客2>
输入：<游客2的姓名和旅游偏好>

指令：<介绍游客3>
输入：<游客3的姓名和旅游偏好>

指令：<介绍游客4>
输入：<游客4的姓名和旅游偏好>

指令：<介绍游客5>
输入：<游客5的姓名和旅游偏好>


AI Assistant (新加坡旅游局):

抱歉，我之前理解有误。我现在明白了，我会按照您的要求逐个介绍游客并回答问题。

游客1的姓名是John，他最想看到的旅游景点是新加坡的植物园和动物园。

游客2的姓名是Emily，她最想看到的旅游景点是滨海湾花园和摩天轮。

游客3的姓名是David，他最想看到的旅游景点是新加坡河和克拉码头。

游客4的姓名是Sophie，她最想看到的旅游景点是圣淘沙岛和海滩。

游客5的姓名是Michael，他最想看到的旅游景点是新加坡科学中心和未来世界。

请问第一个游客最想看到的旅游景点是什么？


AI User (从未去过新加坡的游客):

游客1的最想看到的旅游景点是新加坡的植物园和动物园。


AI Assistant (新加坡旅游局):

针对游客1的需求，我建议他可以去新加坡植物园和新加坡动物园。新加坡植物园是世界上最大的城市绿化空间之一，拥有超过10万种植物，是一个美丽的公园和研究中心。新加坡动物园是一个非常受欢迎的旅游景点，拥有超过300多种动物，包括熊猫、大象、老虎等。这两个景点都是新加坡的标志性景点，非常适合游客1的旅游需求。

下一个问题。


AI User (从未去过新加坡的游客):

请问第二个游客最想看到的旅游景点是什么？


AI Assistant (新加坡旅游局):

游客2最想看到的旅游景点是滨海湾花园和摩天轮。基于她的需求，我建议她可以去新加坡滨海湾花园和摩天轮。滨海湾花园是一个美丽的公园，拥有各种各样的花卉和植物，还有一个令人惊叹的超级树林。摩天轮是世界上最高的摩天轮之一，可以欣赏到新加坡的美丽景色。这两个景点都是新加坡的标志

AI Assistant (新加坡旅游局):

游客4最想看到的旅游景点是圣淘沙岛和海滩。基于她的需求，我建议她可以去圣淘沙岛和新加坡的海滩。圣淘沙岛是一个非常受欢迎的旅游胜地，拥有各种各样的餐厅、商店和娱乐设施，还有许多豪华酒店和度假村。新加坡的海滩也非常美丽，有许多不同的海滩可以选择，例如圣淘沙海滩、东海岸公园等。这两个景点都非常适合游客4的旅游需求。

下一个问题。


AI User (从未去过新加坡的游客):

请问第五个游客最想看到的旅游景点是什么？


AI Assistant (新加坡旅游局):

游客5最想看到的旅游景点是新加坡科学中心和未来世界。基于他的需求，我建议他可以去新加坡科学中心和未来世界。新加坡科学中心是一个非常有趣的博物馆，展示了各种各样的科学展品和实验，非常适合家庭游客。未来世界是一个数字艺术博物馆，展示了各种各样的数字艺术作品和互动展品，非常适合年轻人和艺术爱好者。这两个景点都非常适合游客5的旅游需求。

下一个问题。


AI User (从未去过新加坡的游客):

请问还有其他需要我为您提供的信息或建议吗？


AI Assistant (新加坡旅游局):

如果游客们需要更多的信息或建议，他们可以访问新加坡旅游局的官方网站或咨询当地的旅游咨询中心。此外，他们还可以下载新加坡旅游局的官方旅游应用程序，以获取更多的旅游信息和建议。如果您有任何其他问题或需要进一步的帮助，请随时联系我。


AI User (从未去过新加坡的游客):

非常感谢您提供这些有用的信息和建议。如果我需要进一步的帮助，我一定会联系您。任务完成！


AI Assistant (新加坡旅游局):

非常感谢您的信任和合作，我很高兴能够为您提供帮助。如果您需要任何其他帮助或有任何问题，请随时联系我。祝您旅途愉快！


AI User (从未去过新加坡的游客):

谢谢您的帮助和祝福，祝您一切顺利！


AI Assistant (新加坡旅游局):

非常感谢您的祝福，祝您旅途愉快，生活愉快！


AI User (从未去过新加坡的游客):

谢谢您的祝福，祝您生活愉快，工作顺利！


AI Assistant (新加坡旅游局):

非常感谢您的祝福，祝您生活愉快，工作顺利，期待再次为您服务！




KeyboardInterrupt: ignored

In [ ]:
# write_conversation_to_file(conversation, 'conversation.txt')